# 0.Jupyter常用快捷键

* **Shift + Enter** : 执行当前cell并选中下个cell
* **Shift + J 或 Shift + Down** : 选择下一个cell
* **Shift + K 或 Shift + Up** : 选中上一个cell
* **Shift + M** : 合并选中cell
* **Ctrl + Enter** : 执行下一个cell

* **Enter** ： 编辑模式

* **Esc** : 命令模式
* **Esc a** : 在上方新建cell
* **Esc b** : 在下方新建cell
* **Esc m** : 切换到markdown
* **Esc y** : 切换到code
* **Esc l** : 显示行数
* **Esc d d** : 删除选中的cell
* **Esc 0** : 收起、打开output


# 1.python正则库

In [2]:
# encoding: UTF-8
# py3默认编码utf-8，无需声明
import re
pattern = re.compile(r'hello.*\!')
match = pattern.match('hello, World! how are you?')
if match:
    print(match.group())

hello, World!


# 2.jieba中文处理

代码对 Python 2/3 均兼容

安装：
* 全自动安装：easy_install jieba 或者pip install jieba / pip3 install jieba
* 半自动安装：先下载 http://pypi.python.org/pypi/jieba/ ，解压后运行 python setup.py install
* 手动安装：将 jieba 目录放置于当前目录或者 site-packages 目录

特点：
* 支持三种分词模式：
 * 精确模式，试图将句子最精确地切开，适合文本分析；
 * 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
 * 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
* 支持繁体分词
* 支持自定义词典


## 2.1.基本分词函数与用法

### jieba.cut() & jieba.lcut() 接受三个参数：

* 需要分词的字符串
* cut_all 是否采用全模式
* HMM 是否使用HMM（隐马）模型

### jieba.cut_for_search() & jieba.lcut_for_search() 方法接受两个参数：

* 需要分词的字符串
* 是否使用HMM模型

> * 该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细
> * 待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。注意：不建议直接输入 GBK 字符串，可能无法预料地错误解码成 UTF-8
> * jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)，或者用
jieba.lcut 以及 jieba.lcut_for_search 直接返回 list

### jieba.Tokenizer(dictionary=DEFAULT_DICT) 新建自定义分词器
> 可用于同时使用不同词典。jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。

In [4]:
import jieba

# 全模式
seg_list = jieba.cut("我在学习自然语言处理", cut_all=True)
print("全模式：" + "/".join(seg_list))

# 精准模式(默认)
# set_list = jieba.cut("我在学习自然语言处理")
seg_list = jieba.cut("我在学习自然语言处理", cut_all=False)
print("默认精准模式：" + "/".join(seg_list))

# 搜索引擎模式
seg_list = jieba.cut_for_search("我在学习自然语言处理")
print("搜索引擎模式：" + "/".join(seg_list))


全模式：我/在/学习/自然/自然语言/语言/处理
默认精准模式：我/在/学习/自然语言/处理
搜索引擎模式：我/在/学习/自然/语言/自然语言/处理


## 2.2.用户自定义词典

开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率。
自定义添加词典有两种方式：

1. 用jieba.load_userdict(file_name)加载用户词典，file_name为自定义词典路径；

词典格式如下，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
```
创新办 3 i
云计算 5
凱特琳 nz
台中
```

2. 少量词汇可以自己手动加载
 * add_word(word, freq=None, tag=None) 和 del_word(word)动态添加和删除
 * suggest_freq(segment, tune=True)可调节单个词语的词频，使其能（或不能）被分出来
 * 注意：自动计算的词频在使用 HMM 新词发现功能时可能无效。动态添加词语后分词，需要讲HMM关闭。

In [11]:
import jieba

print("/".join(jieba.cut("如果放到post中将出错", HMM=False)))

# 调整词频使“中将”不被切开
jieba.suggest_freq("中将", True)
print("/".join(jieba.cut("如果放到post中将出错", HMM=False)))


如果/放到/post/中/将/出错
如果/放到/post/中将/出错


## 2.3.关键词提取

### 基于TF-IDF算法的关键词抽取

* jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS={})
 * sentencesentence 为待提取的文本
 * topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
 * withWeight 为是否一并返回关键词权重值，默认值为 False
 * allowPOS 仅包括指定词性的词，默认值为空，即不筛选

* jieba.analyse.TFIDF(idf_path=None) 新建 TFIDF 实例，idf_path 为 IDF 频率文件

In [17]:
import jieba
import jieba.analyse

content = open("西游记.txt", 'rb').read()

# 关键词抽取
tags = jieba.analyse.extract_tags(content, topK=20)
print(",".join(tags))

# 关键词一并返回关键词权重值示例
tags = jieba.analyse.extract_tags(content, topK=20, withWeight=True)
for tag in tags:
    print("{}_{}".format(tag[0], tag[1]))


行者,八戒,师父,三藏,大圣,唐僧,沙僧,菩萨,妖精,和尚,那怪,甚么,那里,长老,呆子,怎么,不知,徒弟,老孙,悟空
行者_0.13416769104125637
八戒_0.06138749884413054
师父_0.055801508802814426
三藏_0.04951909810584418
大圣_0.03438635079555997
唐僧_0.03093405521307161
沙僧_0.028649007279639396
菩萨_0.028405867765903325
妖精_0.025323223636966256
和尚_0.02221375423601498
那怪_0.020860372519413882
甚么_0.020631252946982325
那里_0.019782992142335202
长老_0.018835980952815547
呆子_0.017935812786738954
怎么_0.017029691574655566
不知_0.01675812196592144
徒弟_0.016753840674141294
老孙_0.016216444824348537
悟空_0.01608917368516298


关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径

* 用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
* 自定义语料库示例：https://github.com/fxsjy/jieba/blob/master/extra_dict/idf.txt.big
* 用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_idfpath.py

关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径

* 用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径
* 自定义语料库示例：https://github.com/fxsjy/jieba/blob/master/extra_dict/stop_words.txt
* 用法示例：https://github.com/fxsjy/jieba/blob/master/test/extract_tags_stop_words.py


### 基于TextRank算法的关键词抽取

基于PageRank算法，数据量越大效果越好。

* jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
* jieba.analyse.TextRank() 新建自定义 TextRank 实例

算法论文：[TextRank: Bringing Order into Texts](http://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

基本思想：
1. 将待抽取关键词的文本进行分词
2. 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
3. 计算图中节点的PageRank，注意是无向带权图

In [19]:
import jieba
import jieba.posseg
import jieba.analyse

content = open("西游记.txt", "rb").read()

tags = jieba.analyse.textrank(content, withWeight=True)
for tag in tags:
    print("{}_{}".format(tag[0], tag[1]))

行者_1.0
师父_0.4468256730411676
八戒_0.40251824218027515
三藏_0.37841352093521363
大圣_0.2665149124004022
菩萨_0.23124570809321954
不知_0.20126241891518512
只见_0.1980733274447043
妖精_0.18546445974373882
长老_0.15033785015367465
国王_0.14145894256193234
呆子_0.10492607679999778
徒弟_0.1015174633383265
悟空_0.09583793442461931
不见_0.08953723908347787
不得_0.08825993684157182
小妖_0.08814565706803631
不能_0.0876208317954937
出来_0.08516932481414757
铁棒_0.0735751184496777


## 2.4.词性标注

* jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。
* 标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法

In [21]:
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门")
for word,flag in words:
    print("%s %s" %(word, flag))

我 r
爱 v
北京 ns
天安门 ns


## 2.5.并行分词

### 原理：

将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词，然后归并结果，从而获得分词速度的可观提升

基于 python 自带的 multiprocessing 模块，目前暂不支持 Windows。

### 用法：
jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数
jieba.disable_parallel() # 关闭并行分词模式

注意：并行分词仅支持默认分词器 jieba.dt 和 jieba.posseg.dt。

## 2.6.Tokenize：返回词语在原文起止位置

注意，输入参数只接受 unicode

In [24]:
import jieba

# 默认模式
result = jieba.tokenize(u"我爱北京天安门")
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

# 搜索模式
result = jieba.tokenize(u'永和服装饰品有限公司', mode='search')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

word 我		 start: 0 		 end:1
word 爱		 start: 1 		 end:2
word 北京		 start: 2 		 end:4
word 天安门		 start: 4 		 end:7
word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限		 start: 6 		 end:8
word 公司		 start: 8 		 end:10
word 有限公司		 start: 6 		 end:10


## 2.7.命令行操作

使用示例：python -m jieba news.txt > cut_result.txt

命令行选项（翻译）：

使用: python -m jieba [options] filename

结巴命令行界面。

固定参数:
  filename              输入文件

可选参数:
  -h, --help            显示此帮助信息并退出
  -d [DELIM], --delimiter [DELIM]
                        使用 DELIM 分隔词语，而不是用默认的' / '。
                        若不指定 DELIM，则使用一个空格分隔。
  -p [DELIM], --pos [DELIM]
                        启用词性标注；如果指定 DELIM，词语和词性之间
                        用它分隔，否则用 _ 分隔
  -D DICT, --dict DICT  使用 DICT 代替默认词典
  -u USER_DICT, --user-dict USER_DICT
                        使用 USER_DICT 作为附加词典，与默认词典或自定义词典配合使用
  -a, --cut-all         全模式分词（不支持词性标注）
  -n, --no-hmm          不使用隐含马尔可夫模型
  -q, --quiet           不输出载入信息到 STDERR
  -V, --version         显示版本信息并退出

如果没有指定文件名，则使用标准输入